In [1]:
import datajoint as dj
import os
from faker import Faker
from datetime import datetime

In [ ]:
dj.config["safemode"] = False

In [2]:
fake = Faker()
Faker.seed("sciviz")

In [3]:
dj.conn(
    host=os.getenv("PHARUS_HOST"),
    user=os.getenv("PHARUS_USER"),
    password=os.getenv("PHARUS_PASSWORD"),
)

[2023-02-11 14:38:17,037][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2023-02-11 14:38:17,048][INFO]: Connected root@fakeservices.datajoint.io:3306


DataJoint connection (connected) root@fakeservices.datajoint.io:3306

In [4]:
schema = dj.Schema("demo")
schema.drop()
schema = dj.Schema("demo")

In [5]:
@schema
class Scientist(dj.Lookup):
    definition = """
    scientist_id: int
    ---
    scientist_name: varchar(50)
    """
    contents = [(i, fake.first_name()) for i in range(50)]

In [6]:
@schema
class ExperimentSession(dj.Lookup):
    definition = """
    -> Scientist
    session_datetime: datetime
    """
    contents = [
        (
            i,
            fake.date_time_between(
                start_date=datetime(2022,12,1,0,0,0),
                end_date=datetime(2022,12,31,23,59,59)
            )
        ) for i in range(50) for _ in range(20)
    ]

In [7]:
@schema
class ScientistPerformance(dj.Computed):
    definition = """
    -> Scientist
    ---
    performance_plot: longblob
    """

    def make(self, key):
        scientist_name = (Scientist & key).fetch1("scientist_name")
        datetimes = (ExperimentSession & key).fetch("session_datetime")
        dates = sorted(set(d.date() for d in datetimes))
        session_counts = [sum([dt.date() == d for dt in datetimes]) for d in dates]
        plot = {
            "data": [
                {
                    "x": dates,
                    "y": session_counts,
                    "type": "bar",
                }
            ],
            'layout': {
                'title': f"{scientist_name}'s Session Performance",
                'xaxis': {"title": "Date"},
                'yaxis': {"title": "No. of Sessions"},
            }
        }
        self.insert1(dict(key, performance_plot=plot))

In [9]:
ScientistPerformance.populate()